In [ ]:
import data_utils
import pandas as pd

dataset = pd.read_csv('datasets/dataset_cie10.csv')
display(dataset.head())
dataset.shape

## CIE 10 fields

In [ ]:

def to_year(value_range):
    if(value_range == 'no_cie10'):
        return -1
    
    num = int(value_range[:3])
    unit = value_range[-1]
    
    map_unit = {
        'A': num,
        'M': num / 12,
        'D': num / 365,
        'H': num / 8760
    }
    return int(map_unit[unit])

def get_edad_inf(value):
    value_range = {
        '999': -1,
        'de 0 a 5 años': 0,
        'de 13 a 17 años': 13,
        'de 18 a 24 años': 18,
        'de 25 a 29 años': 25,
        'de 30 a 37 años': 30,
        'de 38 a 49 años': 38,
        'de 50 a 62 años': 50,
        'de 6 a 12 años': 6,
        'mayor de 63 años': 63
            }
    return value_range[value]

def get_edad_sup(value):
    value_range = {
        '999': -1,
        'de 0 a 5 años': 5,
        'de 13 a 17 años': 17,
        'de 18 a 24 años': 24,
        'de 25 a 29 años': 29,
        'de 30 a 37 años': 37,
        'de 38 a 49 años': 49,
        'de 50 a 62 años': 62,
        'de 6 a 12 años': 12,
        'mayor de 63 años': 120
            }
    return value_range[value]

def in_range(row):
    return row['AFEC_EDADR_INF'] >= row['LIMITE_INFERIOR_EDAD_Y'] and  row['AFEC_EDADR_SUP'] < row['LIMITE_SUPERIOR_EDAD_Y']

def cie10_sexo(value):
    if(value == 1 or value == 2):
        return True
    return False


In [ ]:
import numpy as np

cie10_columns = [
    'CAPITULO', 
    'NOMBRE_CAPITULO', 
    'COD_CIE_10_03', 
    'DESCRIPCION_COD_CIE_10_03', 
    'COD_CIE_10_04', 
    'DESCRIPCION_COD_CIE_10_04', 
    'SEXO', 
    'LIMITE_INFERIOR_EDAD', 
    'LIMITE_SUPERIOR_EDAD']

dataset[cie10_columns] = dataset[cie10_columns].replace(np.nan, 'no_cie10', regex=True)
dataset = dataset[dataset['NOMBRE_CAPITULO'] != 'no_cie10']

In [ ]:
for column in dataset.columns:
    dataset[column] = dataset[column].apply(lambda value: '999' if value == 0 or value == '0' or value == 'nan' else value)     

dataset = dataset[dataset['RIESGO_VIDA'] != '999']

In [ ]:
dataset['CIE10_SEXO'] = dataset['SEXO'].apply(cie10_sexo)
dataset['LIMITE_INFERIOR_EDAD_Y'] = dataset['LIMITE_INFERIOR_EDAD'].apply(to_year)
dataset['LIMITE_SUPERIOR_EDAD_Y'] = dataset['LIMITE_SUPERIOR_EDAD'].apply(to_year)
dataset['AFEC_EDADR_INF'] = dataset['AFEC_EDADR'].apply(get_edad_inf)
dataset['AFEC_EDADR_SUP'] = dataset['AFEC_EDADR'].apply(get_edad_sup)
dataset['CIE10_RANGO_EDAD'] = dataset.apply(in_range, axis=1)



## Special case: AFEC_DPTO

States names can take similar names for a same state, and given that there are relatively few states, it's possible to manually fix these values to avoid duplicates in classe values.

Some features are writen in different ways, for example, 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'SAN ANDRES' and 'SAN ANDRÉS' are the same state. Same for 'BOGOTA D.C' and 'BOGOTA D.C.'

In [ ]:
dataset[['AFEC_DPTO']].drop_duplicates().sort_values(by=['AFEC_DPTO'])

In [ ]:
dataset = data_utils.clean_afec_dpto(dataset)

dataset[['AFEC_DPTO']].drop_duplicates().sort_values(by=['AFEC_DPTO'])

In [ ]:

set(dataset['ENT_COD_DEPTO'].values)

### RIESGO_VIDA


In [ ]:

riesgo_vida = dataset['RIESGO_VIDA'].value_counts()
riesgo_vida.plot(kind='bar', title='Patients with life at risk.');

We remove rows with missing info in our target column

In [ ]:
dataset = data_utils.clean_riesgo_vida(dataset)

riesgo_vida = dataset['RIESGO_VIDA'].value_counts()
riesgo_vida.plot(kind='bar', title='Patients with life at risk.');

### CIE_10

In 'Data understanding' notebook we see CIE_10 was way too many missing values. '0' value is the most common value in the column so is not a good candidate for imputing values. But as the column contains descriptions about the patient's illness, we want to keep it as it can provide a signal to predict if the patient's life is at risk.

In [ ]:
dataset = data_utils.clean_cie_10(dataset)
dataset.shape

In [ ]:
riesgo_vida = dataset['RIESGO_VIDA'].value_counts()
riesgo_vida.plot(kind='bar', title='Patients with life at risk.');

Removing records with CIE_10 = 0 reduces drastically the dataset from 2'375.371 to 281.311 records but it provided a huge improvement in the target's balance.

## Removing fields

Acording to the oficial documentation, fields "IDRANGOEDADES", "ID_MES" and "PQR_GRUPOALERTA" have not statistical use, so they are removed from the dataset.

Feature "PQR_ESTADO" has a significant statistical value that may bias the model. Once a PQRS enters the system, it goes through a series of states before the case is closed. First, Historycally, patients with life at risk can have a tendency to have a certain state or a relationship with and another feature (i.e patient's with life at risk may have most of their states as closed as they may have priority over other cases), so including "PQR_ESTADO" will make the model to make predictions over a feature that will not be statiastic relevant when introducing a new PQRS (When a new PQRS enters the system it will have a default state that is very unlikely to have the final state from the original data set).

### Redundant features
These features represent the same data, so we can keep only the codes and loose the descripion.

* COD_MACROMOT = MACROMOTIVO
* COD_MOTGEN = MOTIVO_GENERAL
* COD_MOTESP = MOTIVO_ESPECIFICO
* ENT_COD_DEPTO = ENT_DPTO
* ENT_COD_MPIO = ENT_MPIO
* PET_COD_DEPTO = PET_DPTO

In [ ]:
dataset = data_utils.remove_features(dataset)

display(dataset.head(n = 5))
dataset.shape

## Removing features with more of 75% with empty data


In [ ]:
most_missing_cols = set(dataset.columns[dataset.eq('999').mean() > 0.75])
dataset = dataset.drop(most_missing_cols, axis = 1)
dataset.shape

### Imputing Values: Adding empty mark columns for features with emty values

In [ ]:
set(dataset['ENT_AMBITOIVC'].values)

In [ ]:
dataset['ENT_AMBITOIVC'] = dataset['ENT_AMBITOIVC'].fillna('999')
dataset['ENT_AMBITOIVC'] = dataset['ENT_AMBITOIVC'].apply(lambda value: '999' if value == 'nan' else value)

In [ ]:
set(dataset['IDPATOLOGIA_2'].values)

In [ ]:
dataset['IDPATOLOGIA_2'] = dataset['IDPATOLOGIA_2'].fillna('999')
dataset['IDPATOLOGIA_2'] = dataset['IDPATOLOGIA_2'].apply(lambda value: '999' if value == 'nan' or value == None else value)

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values = '999', strategy="most_frequent")

zero_values = set(dataset.columns[dataset.eq('999').mean() > 0])
for feature in zero_values:
    dataset[f'{feature}_is_missing'] = dataset[feature].apply(lambda f: 1 if f == '999' else 0)
    dataset[feature] = imp.fit_transform(dataset[[feature]])

print(zero_values)
display(dataset['AFEC_DPTO_is_missing'].head())

In [ ]:
dataset.to_csv("datasets/dataset_clean.csv", index = False)

In [ ]:
# Columns with zero values
'''
col_zero_values = set(dataset.columns[dataset.eq('0').mean() > 0])
print(len(col_zero_values))
print(col_zero_values)
dataset = data_utils.impute_values("datasets/dataset_clean.csv", "datasets/dataset_clean_imputed.csv")

'''